# Подготовка к работе: "отпечатки" сцен

Для того, чтобы анализировать, как ведет себя поверхность Земли во времени, удобно составить каталог сцен. Этот каталог представляет собой векторный слой (карту в терминологии GRASS), геометрия в котором -- отпечатки сцен, а атрибутика содержит дату съемки и другую вспомогательную информацию.

Такой каталог позволит по заданной точке быстро получить список сцен и фильтровать их по датам.

## Импорт

Импортируем необходимые модули, а также создадим MAPSET (назовем его footprints), в котором будем работать. По окончании расчетов скопируем полученную карту в PERMANENT и удалим временный MAPSET. 

In [ ]:
import utilites
reload(utilites)

from utilites import (
    get_grassdata_path,
    get_location_name,
    get_location_path,
    format_timestamp,
)

In [ ]:
from grasslib import GRASS

grs = GRASS(gisbase='/usr/lib/grass70', 
            dbase=get_grassdata_path(), 
            location=get_location_name()
)

In [ ]:
grs.grass.run_command('g.mapset', mapset='footprints', flags='c')
mapset = grs.grass.read_command('g.mapset', flags='p')
print mapset

Формируем список сцен:

In [ ]:
scenes = grs.grass.list_strings("group", mapset='landsat', pattern='toar_*')
print scenes

## Создание отпечатка для сцены

Поскольку нам понадобится создать отпечаток для каждой сцены, то будет удобно написать функцию, которая:

 * принимает на вход название группы изображений сцены;
 * формирует метаданные описания сцены (в первую очередь, даты), а также генерирует отпечаток сцены и сохраняет его в виде отдельной карты.
 
Геометрию отпечатка сцены сгенерируем по следующему алгоритму:

1. Выберем первый попавшийся канал из сцены (отпечатки всех каналов плюс-минус одинаковы).
2. Создадим маску по этому каналу (замаскируем все, что null).
3. На базе растра маски создадим временную векторную карту.
4. Генерализуем ее (это не обязательно, но иначе полигон будет иметь много вершин, занимать лишнее место на диске и время обработки);
5. Заполним поля атрибутов.
6. Удалим временные карты.

In [ ]:
def add_footprint(scene, mapset, grass):
    # grass -- объект-обертка над GRASS
    
    year = scene[14: 18]
    day = scene[18:21]

    # Шаг 1
    bandname = scene + '_B1@' + mapset
    # Шаг 2
    try:
        grass.grass.run_command('r.mask', flags='r')
    except:
        pass  # Растра маски не было
    grass.grass.run_command(
        'g.region', raster=bandname, res=10000)  # Точность не нужна
    grass.grass.run_command('r.mask', raster=bandname)
    
    # Шаг 3
    grass.grass.run_command(
        'r.to.vect', 
        input="MASK", output='tmp', type='area', flags='t',
        overwrite=True
    )
    grass.grass.run_command(
        'v.category', input='tmp', output=scene, option='add',
        overwrite=True
    )
    
    # Шаг 4
    # Пропускаем, он вызывает ошибку "Segmentation fault", 
    # нужно отписаться о баге
    
    # Шаг 5  
    grass.grass.run_command(
        'v.db.addtable', map=scene,
        columns="name text, year integer, day integer"
    )
    grass.grass.run_command(
        'v.db.update', map=scene, column='name', value=scene)
    grass.grass.run_command(
        'v.db.update', map=scene, column='year', value=year)
    grass.grass.run_command(
        'v.db.update', map=scene, column='day', value=day)
    
    # Шаг 6
    grass.grass.run_command(
        'g.remove', type='vector', pattern='tmp*', flags='f')
    grass.grass.run_command('r.mask', flags='r')

Прогоним эту функцию в цикле над каждой сценой:

In [ ]:
for scene in scenes:
    scene, mapset = scene.split('@')
    print scene, mapset
    add_footprint(scene, mapset, grs)

print grs.grass.list_strings("vect", mapset='footprints')

## Создание пространственно-временой БД

### Добавление информации о датах

Добавим дату в метаданные к отпечаткам:

In [ ]:
footprints = grs.grass.list_strings("vect", mapset='footprints')
for fp in footprints:
    year, day = grs.grass.read_command(
        'v.db.select', map=fp, columns='year,day', separator=' ', flags='c').split()
    year, day = int(year), int(day)
    stamp = format_timestamp(year, day)
    grs.grass.run_command('v.timestamp', map=fp, date=stamp)

### Создание пространственно-временной БД и регистрация карт

In [ ]:
grs.grass.run_command(
    't.create', type='stvds', temporaltype='absolute', 
    output='landsat_footprints', title="Footprints of Landsat scenes",
    description="Footprints of Landsat scenes", semantictype='mean',
    overwrite=True
)

In [ ]:
for fp in footprints:
    grs.grass.run_command(
        't.register', input='landsat_footprints', maps=fp, type='vector')

In [ ]:
print grs.grass.read_command('t.info', type='stvds', input='landsat_footprints')

## Примеры запросов к БД

### Выборка карт

In [ ]:
print grs.grass.read_command('t.vect.list', input='landsat_footprints')

Выборка по дате. Дата хранится как строковое выражение, поэтому выборка происходит на базе сравнения строк:

In [ ]:
print grs.grass.read_command('t.vect.list', input='landsat_footprints', where="start_time < '2015-08-14'")
print
print grs.grass.read_command('t.vect.list', input='landsat_footprints', where="start_time like '%02%'")

Выборка с преобразованием строки в дату:

In [ ]:
print grs.grass.read_command('t.vect.list', input='landsat_footprints', where="strftime('%m', start_time)='06'")

Более серьезные примеры с использованием временных топологических отношений можно посмотреть [в документации GRASS](https://grass.osgeo.org/grass70/manuals/t.select.html).